## 11.3 词组的两种组织 集合和序列

机器学习模型如何表示单个单词是个相对没有争议的问题,上一节我们提到了如何进行单词的编码.但是另一个棘手的问题: 如何将这些单词组织成句子?

自然语言中单词的顺序是个非常有趣的问题: 与时间序列不同,句子中的单词没有一个自然典型的前后顺序.不同语言下哪怕都是这些单词,其组合顺序相差很大.即使给定一门语言,通常也可以重新排列单词但是表达的意思完全一致.甚至没有语序,只是一堆乱序的单词,我们也能主动的猜测这句话原本的意思,尽管可能出现很多歧义.自然语言句子的语序和意义相关,但又不那么直接.

如何表达词序在不同的 nlp 架构中千差万别,最直接的方法是抛弃词序,将样本转化为无序的单词集合处理.这种方法可以被称为词袋模型.也可以严格按照单词的顺序进行处理,就像处理时序数据一样(这里可以考虑使用 rnn).混合的方式也是可行的,Transformer 架构整体上是不考虑词序的,但是它将词的位置信息注入了它处理的表征中,这使得 Transformer 能同时查看句子的不同部分(与 rnn 不同).因此 Transformer 和 rnn 也被称作序列模型.

历史上看,机器学习早期在 nlp 上的应用只涉及到词袋模型.随着循环神经网络的再次复兴,人们对序列模型的兴趣在 2015 年才逐渐提高，今天这两种模型依然息息相关。

我们将在 imdb 数据集上演示每种方法,在 4 5 章里我们是使用的是预处理完毕矢量化的词袋数据,接下来我们会直接处理 imdb 的原始文本数据.


### 准备 imdb 原始数据

原始数据集在这里 > <https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz>

解压后得到了 aclImdb 文件夹

```text
aclImdb/
...train/
......pos/
......neg/
...test/
......pos/
......neg/
```

- train 是训练数据,test 是测试数据
- pos 是正向数据,neg 是负面数据.
- train/test 各自有 12500,共 25000 条数据.


In [1]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)  #按照种子文件洗牌重排列
    num_val_samples = int(0.2 * len(files))  #取0.2作为验证集
    val_files = files[-num_val_samples:]
    for fname in val_files:  #将验证集移动到单独文件夹
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

将原来的顺序打乱,选取验证集,将验证集移入单独文件夹.


In [3]:
from tensorflow import keras

batch_size = 32  #批次大小

train_ds = keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size)  #训练集
val_ds = keras.preprocessing.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size)  #验证集
test_ds = keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size)  #测试集


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


使用 `text_dataset_from_directory` 构建数据集


In [6]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'What happens when someone has so much social anxiety that they cease to function? How alone can one man get? When the mundane crap we have to do in order to be part of society gets to be too much, what happens? Frownland explores these questions. Definitely a startling original debut from Bronstein. The tone is strange and claustrophobic as we get inside the mind of a guy named Keith that is so messed up he can hardly form a proper sentence. We follow him around as he tries to make contact with people and function day to day. Most of us have known people like this- people that say "sorry" too much or "i appreciate it" when there\'s nothing to appreciate. So we know there are people out there like this but why would someone want to make a movie about them? Well, because its interesting and Bronstein and the lead actor, Dore Mann, do an excellent job. This film 

至此数据准备完成.


## 构建单词集合 词袋方法

最直接处理文本信息的方法,直接忽略顺序,将标记处理成无序的集合.

例子 "the cat sat on the mat" 处理完毕就成了

```text
{"cat", "mat", "on", "sat", "the"}
```

这样做的优点是可以方便的将一句话表示为单一的张量,每一个维度都是一个标记存在不存在.如果使用二进制编码,将一句话表示为一个张量,那么这个张量的维度和集合标记的总数相同,最终的张量中大部分维度都是 0,少数才是 1.(这就是 4 5 章我们使用 imdb 的方式)


In [8]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [9]:
text_vectorization = TextVectorization(
    max_tokens=20000,  #限制最大总标记数量 20000,防止编码空间过大.
    output_mode="binary",  #输出标记为 2进制
)
text_only_train_ds = train_ds.map(lambda x, y: x)  #只有原始文本信息的数据集
text_vectorization.adapt(text_only_train_ds)  # adapt 转换为单词索引表
#张量化数据集
binary_1gram_train_ds = train_ds.map(lambda x, y:
                                     (text_vectorization(x), y))  #训练集
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))  #验证集
binary_1gram_test_ds = test_ds.map(lambda x, y:
                                   (text_vectorization(x), y))  #测试集


使用上一节提到的 TextVectorization 层张量化文本数据.


In [10]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


一个张量有 20000 个维度

每个张量的值完全是 0-1 组成.


In [11]:
from tensorflow import keras
from tensorflow.keras import layers


def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens, ))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)  #全连接层
    x = layers.Dropout(0.5)(x)  #dropout层
    outputs = layers.Dense(1, activation="sigmoid")(x)  #输出层
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

获取 model 的函数,接下来会一直用到.

In [12]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras",
                                    save_best_only=True)  #只保存最好模型
]
model.fit(
    binary_1gram_train_ds.cache(),
    validation_data=binary_1gram_val_ds.cache(),  #验证集
    epochs=10,  #10个epoch
    callbacks=callbacks)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20000)]           0         
_________________________________________________________________
dense (Dense)                (None, 16)                320016    
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 15s 21ms/step - loss: 0.3993 - accuracy: 0.8311 - val_loss: 0.3079 - val_accuracy: 0.8756
Epoch 2/10
625/625 [==============================] - 4s 6ms/step - loss: 0.2733 - accuracy: 0.9007 - val_los

In [13]:
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

782/782 [==============================] - 23s 29ms/step - loss: 0.2925 - accuracy: 0.8875
Test acc: 0.887


我们这里的准确率是 88.7% 相对于基线 50% 已经很好了.
